
# STEP 1: Import ALL the things!


### The packages

In [35]:
DATASET = 'animals'
TEST_IMAGE = 'images/panda.jpg'

# DATASET = 'jcrew'
# TEST_IMAGE = 'images/jc_test.jpeg'

time: 1.18 ms


In [37]:
# !pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.45 ms


In [38]:
import matplotlib
matplotlib.use("Agg")
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

time: 9.22 ms


### The data

In [39]:
## mount your Google Drive folder
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.7 ms


In [41]:
## change into the directory where your data is 
## **** ONLY NEED TO DO THIS ONCE PER RUNTIME ****

# os.chdir("drive/My Drive/data")

time: 851 µs


# STEP 2: Process the data

1. Get image paths. Shuffle them.
2. Loop over the paths and and load the images

Inside the loop (for each image)

* resize the image
* flatten the image
* store the (newly flattened & resized) image in 'data' array
* store the label in 'labels' array

In [42]:
print("[INFO] loading images...")
data = []
labels = []

# imagePaths = sorted(list(paths.list_images('animals')))
imagePaths = sorted(list(paths.list_images(DATASET)))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (32, 32)).flatten()
	data.append(image)
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

[INFO] loading images...
time: 18.6 s


In [43]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

time: 55.2 ms


In [44]:
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

time: 32.8 ms


## Convert the labels from integers to vectors 

NOTE: for 2-class, binary classification you should use Keras' to_categorical function instead as the scikit-learn's LabelBinarizer will not return a
vector

In [45]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

time: 7.36 ms


# STEP 3: MODEL

In [46]:
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

time: 45.9 ms


1. Initialize learning rate and # of epochs
2. Compile the model using:

* OPTIMIZER: SGD
* LOSS: categorical cross-entropy loss

NOTE: use binary_crossentropy for 2-class classification

In [47]:
INIT_LR = 0.01
EPOCHS = 75

print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] training network...
time: 45.8 ms


# STEP 4: Train the neural network

In [48]:
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=EPOCHS, batch_size=32)

Train on 2263 samples, validate on 755 samples
Epoch 1/75
2263/2263 [==============================] - 3s 1ms/step - loss: 1.0965 - acc: 0.3880 - val_loss: 1.0748 - val_acc: 0.4768
Epoch 2/75
2263/2263 [==============================] - 3s 1ms/step - loss: 1.0789 - acc: 0.4034 - val_loss: 1.0870 - val_acc: 0.3298
Epoch 3/75
2263/2263 [==============================] - 3s 1ms/step - loss: 1.0552 - acc: 0.4547 - val_loss: 1.0589 - val_acc: 0.4013
Epoch 4/75
2263/2263 [==============================] - 3s 1ms/step - loss: 1.0383 - acc: 0.4560 - val_loss: 1.0419 - val_acc: 0.4808
Epoch 5/75
2263/2263 [==============================] - 3s 1ms/step - loss: 1.0173 - acc: 0.4954 - val_loss: 1.0222 - val_acc: 0.4927
Epoch 6/75
2263/2263 [==============================] - 3s 1ms/step - loss: 0.9992 - acc: 0.5179 - val_loss: 1.0165 - val_acc: 0.4530
Epoch 7/75
2263/2263 [==============================] - 3s 1ms/step - loss: 0.9865 - acc: 0.5055 - val_loss: 0.9918 - val_acc: 0.5258
Epoch 8/75
2263

# STEP 5: Evaluate the trained network

In [49]:

print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('results.png')

[INFO] evaluating network...
              precision    recall  f1-score   support

        cats       0.47      0.75      0.58       249
        dogs       0.48      0.23      0.31       257
       panda       0.74      0.68      0.71       249

    accuracy                           0.55       755
   macro avg       0.56      0.56      0.53       755
weighted avg       0.56      0.55      0.53       755

time: 411 ms


# STEP 6: Test the model on new data

In [50]:
image = cv2.imread(TEST_IMAGE)
output = image.copy()

## doing the same pre-processing we did above
image = cv2.resize(image, (32, 32))
image = image.astype("float") / 255.0
image = image.flatten()
image = image.reshape((1, image.shape[0]))

time: 265 ms


In [51]:
preds = model.predict(image)

## take the label with the highest probability
i = preds.argmax(axis=1)[0]
label = lb.classes_[i]
print(label, preds[0][i])

panda 0.99365956
time: 7.1 ms
